In [ ]:
import fitz
import numpy
import PyPDF2
import csv
import string
import nltk
nltk.download('punkt') #nltk.tokenize.punkt module.  https://www.nltk.org/_modules/nltk/tokenize/punkt.html
nltk.download('averaged_perceptron_tagger') #nltk.tag.perceptron module, https://explosion.ai/blog/part-of-speech-pos-tagger-in-python
nltk.download('maxent_ne_chunker') #https://www.kaggle.com/datasets/nltkdata/maxent-ne-chunker , https://www.tomaarsen.com/nltk/api/nltk.chunk.named_entity.html
nltk.download('words') #list of str https://www.nltk.org/api/nltk.corpus.reader.html?highlight=words#nltk.corpus.reader.AlignedCorpusReader.words
from nltk.tokenize import sent_tokenize

from nltk.corpus import stopwords
nltk.download('stopwords')
stoplist = stopwords.words('english')
stopset = set(stopwords.words('english'))

import re
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher 
from spacy.tokens import Span 

import networkx as nx

import matplotlib.pyplot as plt
from tqdm import tqdm

pd.set_option('display.max_colwidth', 200)
%matplotlib inline

In [ ]:
def preprocess(pdf):

    #sent_seg=sent_tokenize(pdf)
    sent_seg = nlp(pdf)
    for sentence in sent_seg.sents:
        print("Segmentierte Sätze", sentence)
       
    #print("Segmentierte Sätze\n", sent_seg_sep, "\n")

#tokens=[word.lower() for word in tokens if word.isalpha()]
   # pdf_ent = nlp(pdf)
   # print("named entities\n")
   # for ent in pdf_ent.ents:
      #  print(ent.text, ent.start_char, ent.end_char, ent.label_)
        
   # tokens = nltk.word_tokenize(pdf)
   #tokens = [''.join(letter for letter in word if letter not in string.punctuation) for word in tokens if word]
#print(tokens)

# Herasufiltern Stop words
   # filtered_sent = []
  #  for w in tokens:
    #    if w not in stopset:
   #         filtered_sent.append(w)
   # print("\n Sätze ohne Stopwords\n", filtered_sent)

    #exmpl_postagged = nltk.pos_tag(filtered_sent)
   # print("\n Sätze Postagged\n", exmpl_postagged)

    #exmpl_chunks = nltk.ne_chunk(exmpl_postagged)
   # print("\n Chunks\n", exmpl_chunks)

    return(sent_seg)

def get_entities(sent):
  ## chunk 1
  ent1 = ""
  ent2 = ""

  prv_tok_dep = ""    # dependency tag of previous token in the sentence
  prv_tok_text = ""   # previous token in the sentence

  prefix = ""
  modifier = ""

  #############################################################
  
  for tok in nlp(sent):
    ## chunk 2
    # if token is a punctuation mark then move on to the next token
    if tok.dep_ != "punct":
      # check: token is a compound word or not
      if tok.dep_ == "compound":
        prefix = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text
      
      # check: token is a modifier or not
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text
      
      ## chunk 3
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      ## chunk 4
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix +" "+ tok.text
        
      ## chunk 5  
      # update variables
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text
  #############################################################

  return [ent1.strip(), ent2.strip()]

def get_relation(sent):
    doc = nlp(sent)
    # Matcher class object
    matcher = Matcher(nlp.vocab)
    relation=[]
    # define the pattern
    pattern = [{'DEP': 'ROOT'},
               {'DEP': 'prep', 'OP': "?"},
               {'DEP': 'agent', 'OP': "?"},
               {'POS': 'ADJ', 'OP': "?"}]

    matcher.add("matching_1", [pattern], on_match=None)

    matches = matcher(doc)
  
    for mathc_id, start, end in matches:
        matched_span = doc[start: end]
        # print(matched_span.text)
        relation.append(matched_span.text)
    return relation

In [ ]:
# extract subject
source = [i[0] for i in entity_pairs]

# extract object
target = [i[1] for i in entity_pairs]

kg_df = pd.DataFrame({'source':source, 'target':target, 'edge':relations})
kg_df

In [ ]:
sent_seg=preprocess(text)
print(sent_seg)

#zu csv und schreiben
#df_csv = pd.DataFrame(data={"sentence": sent_seg})
#print(df_csv)
#df_csv.to_csv("C:/Users/Jana/OneDrive/Dokumente/Desktop/Uni/Master/Thesis/Test/sentence_as_csv.csv", sep=';',index=False)

#print(df_csv.shape)
#df_csv['sentence'].sample(5)

In [ ]:
entity_pairs = []

for i in tqdm(df_csv["sentence"]):
  entity_pairs.append(get_entities(i))

In [ ]:
entity_pairs[10:20]

In [ ]:
relations = [get_relation(i) for i in tqdm(df_csv['sentence'])]

In [ ]:
pd.Series(relations).value_counts()[:50]